# IBM Data Science Capstone Project

Note: this notebook is dedicated for the IBM Data Science certificate capstone project

### Week 3 - Segmenting and Clustering Neighborhoods in Toronto

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml

import pandas as pd
import numpy as np

Webscrapping - wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
html = r.text
bs = BeautifulSoup(html)
#print(bs.prettify())  Check out the prettified html

In [3]:
result =[]
for item in bs.find_all('td'):
    result.append(item.get_text())

Data cleaning

In [4]:
#remove unwanted items
result=result[:-33]

In [5]:
postal_code=result[::3]
postal_code = [x[:-1] for x in postal_code]

In [6]:
borough = result[1::3]
borough = [x[:-1] for x in borough]

In [7]:
neighborhood = result[2::3]
neighborhood = [x[:-1] for x in neighborhood]

In [8]:
data = {'PostalCode': postal_code, 'borough': borough, 'neighborhood': neighborhood}
df = pd.DataFrame(data, columns=data.keys())

In [13]:
df.head()

,PostalCode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
#ignore rows without an assigned borough
df = df[df.borough!='Not assigned']
df.reset_index(inplace=True, drop=True)
df.head()

,PostalCode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#check if there are any 'Not assigned' neighborhood
sum(df.neighborhood=='Not assigned')

0

In [14]:
df.shape

(103, 3)